### aim: 

### date: 

In [10]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [11]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [12]:
import numpy as np

import netCDF4 as nc
import xarray as xr
import sys

import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

import glob
import pandas as pd


###



In [38]:
regs = ['ARCTIC', 'P1', 'P2', 'P3', 'P4', 'P5', 'A1', 'A2', 'A3', 'A4', 'A5', 'I3', 'I4', 'I5']

regdict = {'ARCTIC' : {'number' : 0.5},
           'P1' : {'number': 1.0},
          'P2' : {'number': 1.2},
           'P3' : {'number': 1.4},
           'P4' : {'number': 1.6},
           'P5' : {'number': 1.8},
            'A1' : {'number': 2.4},
          'A2' : {'number': 2.6},
           'A3' : {'number': 2.8},
           'A4' : {'number': 3},
           'A5' : {'number': 3.2},
           'I3' : {'number': 3.6},
           'I4' : {'number': 3.8},
           'I5' : {'number': 4},
           
          }
    
tics = []
tcm = 'Spectral'
tmask = nc.Dataset('/gpfs/data/greenocean/software/resources/breakdown/clq_basin_masks_ORCA.nc')

maskno = np.zeros([149,182])
for i in range(0, len(regs)):
    maskno[tmask[regs[i]][:] == 1] = regdict[regs[i]]['number']
    tics.append(regdict[regs[i]]['number'])
maskno[maskno == 0] = np.nan

mask_latbands = np.copy(maskno)

mask_latbands[maskno == 0.5] = 1
mask_latbands[(maskno == 1) | (maskno == 2.4)] = 2
mask_latbands[(maskno == 1.2) | (maskno == 2.6)] = 3
mask_latbands[(maskno == 1.4) | (maskno == 2.8) | (maskno == 3.6)] = 4
mask_latbands[(maskno == 1.6) | (maskno == 3.0) | (maskno == 3.8)] = 5
mask_latbands[(maskno == 1.8) | (maskno == 3.2) | (maskno == 4.0)] = 6

#####
tmesh = xr.open_dataset('/gpfs/data/greenocean/software/resources/regrid/mesh_mask3_6.nc')
tmesh['csize'] = tmesh.tmask[0,0,:,:] * tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]

csize_broad = np.zeros([31,149,182])
for i in range(0,31):
    csize_broad[i,:,:] = tmesh['csize'].values
cvol = csize_broad * tmesh['e3t_0'][0,:,:,:] * tmesh['tmask'][0,:,:,:]

cdepth = tmesh['e3t_0'][0,:,:,:] * tmesh['tmask'][0,:,:,:]
cdepth_broad = np.zeros([12,31,149,182])
for i in range(0,12):
    cdepth_broad[i,:,:,:] = cdepth
    
def make_yearlist(yrst, yrend, dtype, tr, baseDir = '/gpfs/afm/greenocean/software/runs'):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        ty = f'{baseDir}/{tr}/ORCA2_1m_{yrs[i]}*{dtype}*.nc'
        t2 = glob.glob(ty)
        #print(t2)
        ylist.append(t2[0])
    return ylist

def gimme_var(ds, var):
    '''
    takes ds of type grid, calculates seasonal by-lat 1-d tracers (tos, sos, lmdr10_1.
    '''
    seas_lat_var = np.zeros([12,6]) #storage array
    #print(ds[plant].long_name)
    tmn = ds[var] #.groupby('time_counter.season').mean('time_counter') #get seasonal mean1
    tmesh['csize'] = tmesh.tmask[0,0,:,:] #* tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]
    tmn_molm2 = tmn*tmesh.csize #just 0,1
    #tmn_molm2 = np.nansum(tmn_test, axis = 1) #integrate, ie sum

    for s in range(0,12):        
        for b in range(0,6):
            tmn_molm2_formask = np.copy(tmn_molm2) #copy for manipulations
            tmn_molm2_formask_tseas = tmn_molm2_formask[s,:,:] # 
            tmn_molm2_formask_tseas[mask_latbands != b+1] = 0
            csize_formask = np.copy(csize_broad[0,:,:])
            csize_formask[mask_latbands != b+1] = 0
            seas_lat_var[s,b] = np.ma.average(tmn_molm2_formask_tseas[:,:], weights=csize_formask)

    return seas_lat_var

In [40]:
def make_tracer_seas_lat_gridT(yrst, tr, baseDir = '/gpfs/afm/greenocean/software/runs/'):
    
    tdir = '/gpfs/home/mep22dku/scratch/SOZONE/ecosystemLooks/mld-and-friends/'
    tnam1 = f'{tr}-gridT-{yrst}.nc'
    tnam = f'{tdir}{tr}-gridT-{yrst}.nc'
    print(tnam1)
    
    dtype = 'grid_T'
    #baseDir = '/gpfs/afm/greenocean/software/runs/'
    yrend = yrst #
    ylist = make_yearlist(yrst, yrend, dtype, tr, baseDir)
    ds = xr.open_dataset(ylist[0])
    
    times = pd.date_range(f"{yrst}/01/01",f"{yrst+1}/01/01",freq='MS',closed='left')

    seas_lat_tos = gimme_var(ds, 'tos')
    seas_lat_sos = gimme_var(ds, 'sos')
    seas_lat_mldr10_1 = gimme_var(ds, 'mldr10_1')
    
    data_vars = {
#                     'Alkalini':(['time_counter',  'lat_band'], seas_lat_Alkalini,
#                  {'units': 'mol/m2'}),
                     
                 'sos':(['time_counter',  'lat_band'], seas_lat_sos,
                 {'units': 'g/kg'}),
                 'tos':(['time_counter',  'lat_band'], seas_lat_tos,
                 {'units': 'degC'}),
                 'mldr10_1':(['time_counter',  'lat_band'], seas_lat_mldr10_1,
                 {'units': 'm'}),        

                }

    # define coordinates
    coords = {'time_counter': (['time_counter'], times),\
             'lat_band': (['lat_band'], np.arange(1,7,1)),}

    # define global attributes
    attrs = {'made in':'SOZONE/ecosystemLooks/fxlgroups-intime.ipynb',
             'latitude bands': 'see notebook, 1 is arctic, 6 is SO',
            }
    
#     # create dataset
    ds2 = xr.Dataset(data_vars=data_vars,
                    coords=coords,
                    attrs=attrs)
    
    ds2.to_netcdf(f'{tnam}')
    
for i in range(1950,2100):
    make_tracer_seas_lat_gridT(i, 'TOM12_TJ_1ASA')
    make_tracer_seas_lat_gridT(i, 'TOM12_TJ_1BSA')
    make_tracer_seas_lat_gridT(i, 'TOM12_TJ_1AA6')
    make_tracer_seas_lat_gridT(i, 'TOM12_TJ_1BA6')

TOM12_TJ_1ASA-gridT-1950.nc
TOM12_TJ_1BSA-gridT-1950.nc
TOM12_TJ_1AA6-gridT-1950.nc
TOM12_TJ_1BA6-gridT-1950.nc
TOM12_TJ_1ASA-gridT-1951.nc
TOM12_TJ_1BSA-gridT-1951.nc
TOM12_TJ_1AA6-gridT-1951.nc
TOM12_TJ_1BA6-gridT-1951.nc
TOM12_TJ_1ASA-gridT-1952.nc
TOM12_TJ_1BSA-gridT-1952.nc
TOM12_TJ_1AA6-gridT-1952.nc
TOM12_TJ_1BA6-gridT-1952.nc
TOM12_TJ_1ASA-gridT-1953.nc
TOM12_TJ_1BSA-gridT-1953.nc
TOM12_TJ_1AA6-gridT-1953.nc
TOM12_TJ_1BA6-gridT-1953.nc
TOM12_TJ_1ASA-gridT-1954.nc
TOM12_TJ_1BSA-gridT-1954.nc
TOM12_TJ_1AA6-gridT-1954.nc
TOM12_TJ_1BA6-gridT-1954.nc
TOM12_TJ_1ASA-gridT-1955.nc
TOM12_TJ_1BSA-gridT-1955.nc
TOM12_TJ_1AA6-gridT-1955.nc
TOM12_TJ_1BA6-gridT-1955.nc
TOM12_TJ_1ASA-gridT-1956.nc
TOM12_TJ_1BSA-gridT-1956.nc
TOM12_TJ_1AA6-gridT-1956.nc
TOM12_TJ_1BA6-gridT-1956.nc
TOM12_TJ_1ASA-gridT-1957.nc
TOM12_TJ_1BSA-gridT-1957.nc
TOM12_TJ_1AA6-gridT-1957.nc
TOM12_TJ_1BA6-gridT-1957.nc
TOM12_TJ_1ASA-gridT-1958.nc
TOM12_TJ_1BSA-gridT-1958.nc
TOM12_TJ_1AA6-gridT-1958.nc
TOM12_TJ_1BA6-gridT-

TOM12_TJ_1BSA-gridT-2023.nc
TOM12_TJ_1AA6-gridT-2023.nc
TOM12_TJ_1BA6-gridT-2023.nc
TOM12_TJ_1ASA-gridT-2024.nc
TOM12_TJ_1BSA-gridT-2024.nc
TOM12_TJ_1AA6-gridT-2024.nc
TOM12_TJ_1BA6-gridT-2024.nc
TOM12_TJ_1ASA-gridT-2025.nc
TOM12_TJ_1BSA-gridT-2025.nc
TOM12_TJ_1AA6-gridT-2025.nc
TOM12_TJ_1BA6-gridT-2025.nc
TOM12_TJ_1ASA-gridT-2026.nc
TOM12_TJ_1BSA-gridT-2026.nc
TOM12_TJ_1AA6-gridT-2026.nc
TOM12_TJ_1BA6-gridT-2026.nc
TOM12_TJ_1ASA-gridT-2027.nc
TOM12_TJ_1BSA-gridT-2027.nc
TOM12_TJ_1AA6-gridT-2027.nc
TOM12_TJ_1BA6-gridT-2027.nc
TOM12_TJ_1ASA-gridT-2028.nc
TOM12_TJ_1BSA-gridT-2028.nc
TOM12_TJ_1AA6-gridT-2028.nc
TOM12_TJ_1BA6-gridT-2028.nc
TOM12_TJ_1ASA-gridT-2029.nc
TOM12_TJ_1BSA-gridT-2029.nc
TOM12_TJ_1AA6-gridT-2029.nc
TOM12_TJ_1BA6-gridT-2029.nc
TOM12_TJ_1ASA-gridT-2030.nc
TOM12_TJ_1BSA-gridT-2030.nc
TOM12_TJ_1AA6-gridT-2030.nc
TOM12_TJ_1BA6-gridT-2030.nc
TOM12_TJ_1ASA-gridT-2031.nc
TOM12_TJ_1BSA-gridT-2031.nc
TOM12_TJ_1AA6-gridT-2031.nc
TOM12_TJ_1BA6-gridT-2031.nc
TOM12_TJ_1ASA-gridT-

TOM12_TJ_1AA6-gridT-2096.nc
TOM12_TJ_1BA6-gridT-2096.nc
TOM12_TJ_1ASA-gridT-2097.nc
TOM12_TJ_1BSA-gridT-2097.nc
TOM12_TJ_1AA6-gridT-2097.nc
TOM12_TJ_1BA6-gridT-2097.nc
TOM12_TJ_1ASA-gridT-2098.nc
TOM12_TJ_1BSA-gridT-2098.nc
TOM12_TJ_1AA6-gridT-2098.nc
TOM12_TJ_1BA6-gridT-2098.nc
TOM12_TJ_1ASA-gridT-2099.nc
TOM12_TJ_1BSA-gridT-2099.nc
TOM12_TJ_1AA6-gridT-2099.nc
TOM12_TJ_1BA6-gridT-2099.nc


In [ ]:
def make_yearlist(yrst, yrend, tr):
    baseDir = '/gpfs/home/mep22dku/scratch/SOZONE/ecosystemLooks/mld-and-friends'
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        ty = f'{baseDir}/{tr}/ORCA2_1m_{yrs[i]}*{dtype}*.nc'
        t2 = glob.glob(ty)
        #print(t2)
        ylist.append(t2[0])
    return ylist

In [41]:
for i in range(1950,2023):
    make_tracer_seas_lat_gridT(i, 'TOM12_TJ_RVA0')
    make_tracer_seas_lat_gridT(i, 'TOM12_ET_SI73', baseDir = '/gpfs/home/e031/scratch/')

TOM12_TJ_RVA0-gridT-1950.nc
TOM12_ET_SI73-gridT-1950.nc
TOM12_TJ_RVA0-gridT-1951.nc
TOM12_ET_SI73-gridT-1951.nc
TOM12_TJ_RVA0-gridT-1952.nc
TOM12_ET_SI73-gridT-1952.nc
TOM12_TJ_RVA0-gridT-1953.nc
TOM12_ET_SI73-gridT-1953.nc
TOM12_TJ_RVA0-gridT-1954.nc
TOM12_ET_SI73-gridT-1954.nc
TOM12_TJ_RVA0-gridT-1955.nc
TOM12_ET_SI73-gridT-1955.nc
TOM12_TJ_RVA0-gridT-1956.nc
TOM12_ET_SI73-gridT-1956.nc
TOM12_TJ_RVA0-gridT-1957.nc
TOM12_ET_SI73-gridT-1957.nc
TOM12_TJ_RVA0-gridT-1958.nc
TOM12_ET_SI73-gridT-1958.nc
TOM12_TJ_RVA0-gridT-1959.nc
TOM12_ET_SI73-gridT-1959.nc
TOM12_TJ_RVA0-gridT-1960.nc
TOM12_ET_SI73-gridT-1960.nc
TOM12_TJ_RVA0-gridT-1961.nc
TOM12_ET_SI73-gridT-1961.nc
TOM12_TJ_RVA0-gridT-1962.nc
TOM12_ET_SI73-gridT-1962.nc
TOM12_TJ_RVA0-gridT-1963.nc
TOM12_ET_SI73-gridT-1963.nc
TOM12_TJ_RVA0-gridT-1964.nc
TOM12_ET_SI73-gridT-1964.nc
TOM12_TJ_RVA0-gridT-1965.nc
TOM12_ET_SI73-gridT-1965.nc
TOM12_TJ_RVA0-gridT-1966.nc
TOM12_ET_SI73-gridT-1966.nc
TOM12_TJ_RVA0-gridT-1967.nc
TOM12_ET_SI73-gridT-

IndexError: list index out of range

In [15]:
fact = 0.9
fig, axs = plt.subplots(6,2, figsize=(15*fact, 17*fact), facecolor='w', edgecolor='k')
axs = axs.ravel()

In [16]:
ds = make_tracer_seas_lat_gridT(1950, 'TOM12_TJ_1AA6', baseDir = '/gpfs/afm/greenocean/software/runs/')

//gpfs/home/mep22dku/scratch/SOZONE/ecosystemLooks/mld-and-friends/TOM12_TJ_1AA6-grid-1950.nc


In [33]:
# ds.tos
# ds.sos
# ds.mldr10_1


def gimme_plant(ds, var):
    '''
    takes ds of type grid, calculates seasonal by-lat 1-d tracers (tos, sos, lmdr10_1.
    '''
    seas_lat_var = np.zeros([12,6]) #storage array
    #print(ds[plant].long_name)
    tmn = ds[var] #.groupby('time_counter.season').mean('time_counter') #get seasonal mean1
    tmesh['csize'] = tmesh.tmask[0,0,:,:] #* tmesh.e1t[0,:,:] * tmesh.e2t[0,:,:]
    tmn_molm2 = tmn*tmesh.csize #just 0,1
    #tmn_molm2 = np.nansum(tmn_test, axis = 1) #integrate, ie sum

    for s in range(0,12):        
        for b in range(0,6):
            tmn_molm2_formask = np.copy(tmn_molm2) #copy for manipulations
            tmn_molm2_formask_tseas = tmn_molm2_formask[s,:,:] # 
            tmn_molm2_formask_tseas[mask_latbands != b+1] = 0
            csize_formask = np.copy(csize_broad[0,:,:])
            csize_formask[mask_latbands != b+1] = 0
            seas_lat_var[s,b] = np.ma.average(tmn_molm2_formask_tseas[:,:], weights=csize_formask)

    return seas_lat_var

seas_lat_var = gimme_plant(ds,'mldr10_1')

In [36]:
print(seas_lat_var[:,1])

[76.5935682  96.97762168 96.57737209 73.84660715 27.72591793 19.21026259
 18.7983233  18.54187459 21.192777   26.28483934 34.46279294 45.04934174]
